In [1]:
from openpyxl import load_workbook
import pandas as pd
from datetime import time, timedelta
from scheduler import Optimizer, ROLE_NAME, Member, Task
import pytz
from typing import Tuple, List, Annotated, Literal, Optional, Any, Union


load C:\Users\nicho\AppData\Roaming\Python\Python313\site-packages\ortools\.libs\zlib1.dll...
load C:\Users\nicho\AppData\Roaming\Python\Python313\site-packages\ortools\.libs\abseil_dll.dll...
load C:\Users\nicho\AppData\Roaming\Python\Python313\site-packages\ortools\.libs\utf8_validity.dll...
load C:\Users\nicho\AppData\Roaming\Python\Python313\site-packages\ortools\.libs\re2.dll...
load C:\Users\nicho\AppData\Roaming\Python\Python313\site-packages\ortools\.libs\libprotobuf.dll...
load C:\Users\nicho\AppData\Roaming\Python\Python313\site-packages\ortools\.libs\highs.dll...
load C:\Users\nicho\AppData\Roaming\Python\Python313\site-packages\ortools\.libs\ortools.dll...


### Generate schedule from Excel

#### Get all members and tasks from the excel file

In [2]:
def get_members_and_tasks(excel_file=r"C:\Users\nicho\OneDrive - HKUST Connect\Python\Github\production_scheduler\wilson\Member&Task_Info.xlsx") -> Tuple[List[Task], List[Member]]:

  sheet_name = "Members_availability"
  workbook = load_workbook(filename=excel_file, data_only=True)
  sheet = workbook['Members']
  sheet2 = workbook['Tasks']

  member_names = []
  roles = []
  member_hr_rates = []
  member_ot_rates = []
  members = []
  tasks = []

  for cell in sheet[3][1:]:
      if cell.value:
          member_names.append(cell.value)

          col_index = cell.col_idx
          roles.append(sheet.cell(row=4, column=col_index).value)
          member_hr_rates.append(sheet.cell(row=5, column=col_index).value)
          member_ot_rates.append(sheet.cell(row=6, column=col_index).value)

  member_roles = [key for string in roles for key, value in ROLE_NAME.items() if value == string]

  def format_time(row_index):
      hours = row_index // 4
      minutes = (row_index % 4) * 15
      if hours == 24 and minutes == 0:
          return 23, 59
      else:
          return hours, minutes

  def process_unavailability(file_path, sheet_name="Members_availability"):
      df = pd.read_excel(file_path, sheet_name=sheet_name, header=None)
      print(df.to_markdown())
      times = df.iloc[:, 0]
      results = {}
      for col_idx in range(1, df.shape[1]): 
          availability = df.iloc[:, col_idx].values 
          intervals = []
          start = None
          for i, available in enumerate(availability):
              if available == 0 and start is None:
                  start = i 
              elif available == 1 and start is not None:
                  intervals.append((format_time(start-1), format_time(i-1)))
                  start = None
          if start is not None:
              intervals.append((format_time(start-1), format_time(len(availability)-1)))
          results[f"Person_{col_idx}"] = intervals
      return results

  unavailability_results = process_unavailability(excel_file, sheet_name)
  member_intervals = []
  for person, intervals in unavailability_results.items():
      member_intervals.append(intervals)

  for i, name in enumerate(member_names):
      converted_intervals = [
          (time(start[0], start[1]), time(end[0], end[1])) for start, end in member_intervals[i]]
      members.append(Member(id=i, name=name, rate=member_hr_rates[i], ot=member_ot_rates[i], role=member_roles[i], blocked_timeslots=converted_intervals))

  task_names = []
  task_locations = []
  task_durations = []
  task_dependencies = []
  # task_timeofday = []
  task_actors = []
  task_roles = []

  for row in range(3, sheet2.max_row+1):
      cell_value = sheet2.cell(row=row, column=3).value
      if cell_value:
          task_names.append(cell_value)
          task_locations.append(sheet2.cell(row=row, column = 9).value)
          task_durations.append(sheet2.cell(row=row, column = 10).value)
          task_dependencies.append(sheet2.cell(row=row, column = 12).value)
          task_actors.append(sheet2.cell(row=row, column = 16).value)
          task_roles.append(sheet2.cell(row=row, column = 25).value)

  for i, name in enumerate(task_names):
      task_location = eval(task_locations[i])
      task_duration = task_durations[i]
      task_actor = eval(task_actors[i])
      task_role = eval(task_roles[i])
      task_dependency = eval(task_dependencies[i])
      tasks.append(Task(id=i, description=name, location=task_location, estimated_duration=task_duration, members=task_actor, roles=task_role, dependencies=task_dependency))
  
  return tasks, members

#### Generate schedule

In [3]:
def generate_schedule(
    excel_file=r"C:\Users\nicho\OneDrive - HKUST Connect\Python\Github\production_scheduler\wilson\Member&Task_Info.xlsx",
    output_location: str = r'C:\Users\nicho\OneDrive - HKUST Connect\Python\Github\production_scheduler\nicholas\output',
    timeout: int = 60,
):
  tasks, members = get_members_and_tasks(excel_file=excel_file)
  model = Optimizer(tasks, members)
  res = model.schedule_tasks(timeout=timeout)
  model.export_schedule(output_location)

  result = {"cost": model.optimized_cost}
        
  return result

In [4]:
generate_schedule(excel_file=r"I:\Member^0Task_Info.xlsx")

|    | 0            | 1     | 2     | 3      | 4   | 5    | 6    | 7   | 8     | 9     | 10     | 11     | 12    | 13     | 14     | 15    | 16      | 17      | 18      | 19      | 20      |
|---:|:-------------|:------|:------|:-------|:----|:-----|:-----|:----|:------|:------|:-------|:-------|:------|:-------|:-------|:------|:--------|:--------|:--------|:--------|:--------|
|  0 | Availability | Jacob | Vicky | Monita | Ray | John | Sara | Raj | Kumar | Rahul | Rajesh | Ramesh | Rajat | Bonnie | Sheila | Sandy | Camera1 | Camera2 | Camera3 | Camera4 | Camera5 |
|  1 | 00:00:00     | 0     | 0     | 0      | 0   | 0    | 0    | 0   | 0     | 0     | 0      | 0      | 0     | 0      | 0      | 0     | 0       | 1       | 0       | 0       | 0       |
|  2 | 00:15:00     | 0     | 0     | 0      | 0   | 0    | 0    | 0   | 0     | 0     | 0      | 0      | 0     | 0      | 0      | 0     | 0       | 1       | 0       | 0       | 0       |
|  3 | 00:30:00     | 0     | 0     | 0      

{'cost': 30274.0}

### Dynamic schedulling

In [25]:
def filter_schedule(
    members: Optional[Annotated[List[str], 'List of members to be selected.']]=None,
    tasks: Optional[Annotated[List[int], 'List of tasks ids to be selected.']]=None,
    start_time: Optional[Annotated[Tuple[time, Literal['ge', 'le', 'gt', 'lt', 'eq']], 'Filter tasks with start time different operator. Time ranges from 00:00 to 23:59. Note that no tasks will start before 00:00.']]=None,
    end_time: Optional[Annotated[Tuple[time, Literal['ge', 'le', 'gt', 'lt', 'eq']], 'Filter tasks with end time with different operator.  Time ranges from 00:00 to 23:59. Note that no tasks will end after 23:59.']]=None,
    duration: Optional[Annotated[Tuple[int, Literal['ge', 'le', 'gt', 'lt', 'eq']], 'Filter tasks with a duration that fits the duration requirement.']]=None,
    output_type: Optional[Literal['markdown','dataframe']]='markdown',
    schedule_file_path: str = r'C:\Users\nicho\OneDrive - HKUST Connect\Python\Github\production_scheduler\nicholas\coding\schedule_solution_1.csv') -> str:

    schedule_file_path = r'C:\Users\nicho\OneDrive - HKUST Connect\Python\Github\production_scheduler\nicholas\coding\schedule_solution_1.csv'
    df = pd.read_csv(schedule_file_path)
    df['Start'] = pd.to_datetime(df['Start']).dt.time
    df['End'] = pd.to_datetime(df['End']).dt.time

    if members:
        df = df[df['Member'].isin(members)]
    if tasks:
        df = df[df['Task'].isin(str(tasks))]
    if start_time:
      if start_time[1] =='ge':
        df = df[df['Start'] >= start_time[0]]
      elif start_time[1] == 'le':
        df = df[df['Start'] <= start_time[0]]
      elif start_time[1] == 'gt':
        df = df[df['Start'] > start_time[0]]
      elif start_time[1] == 'lt':
        df = df[df['Start'] < start_time[0]]
      elif start_time[1] == 'eq':
        df = df[df['Start'] == start_time[0]]
    if end_time:
      if end_time[1] == 'ge':
        df = df[df['End'] >= end_time[0]]
      elif end_time[1] == 'le':
        df = df[df['End'] <= end_time[0]]
      elif end_time[1] == 'gt':
        df = df[df['End'] > end_time[0]]
      elif end_time[1] == 'lt':
        df = df[df['End'] < end_time[0]]
      elif end_time[1] == 'eq':
        df = df[df['End'] == end_time[0]]

    if duration:
      if duration[1] == 'ge':
        df = df[df['Duration'] >= duration[0]]
      elif duration[1] == 'le':
        df = df[df['Duration'] <= duration[0]]
      elif duration[1] == 'gt':
        df = df[df['Duration'] > duration[0]]
      elif duration[1] == 'lt':
        df = df[df['Duration'] < duration[0]]
      elif duration[1] == 'eq':
        df = df[df['Duration'] == duration[0]]

    return df.to_markdown(index=False) if output_type == 'markdown' else df


def dynamic_schedulling(
    members: Optional[Annotated[List[Member], 'List of members to be updated.']]=[],
    tasks: Optional[Annotated[List[Task], 'List of tasks to be selected.']]=[],
    schedule_file: Optional[Annotated[str, 'The location of the schedule.']] = r'C:\Users\nicho\OneDrive - HKUST Connect\Python\Github\production_scheduler\nicholas\coding\schedule_solution_1.csv',
    timeout: Optional[Annotated[int, 'Timeout for the optimization.']]=60,
    output_location: str = r'C:\Users\nicho\OneDrive - HKUST Connect\Python\Github\production_scheduler\nicholas\output',
    current_time: Optional[Annotated[time, 'Current time.']]=None,
) -> int:

  all_tasks, all_members = get_members_and_tasks()

  # Get current time
  now_china = current_time
  if now_china is None:
    now_utc = datetime.now(pytz.utc)
    china_tz = pytz.timezone('Asia/Shanghai')
    now_china = now_utc.astimezone(china_tz).time()
    now_china = time(now_china.hour, now_china.minute)

  # Replace members in all_members with matching IDs from members list
  for new_member in members:
      for i, existing_member in enumerate(all_members):
          if existing_member.id == new_member.id:
              all_members[i] = new_member
  
  for new_task in tasks:
    for i, existing_task in enumerate(all_tasks):
        if existing_task.id == new_task.id:
            all_tasks[i] = new_task

  df = pd.read_csv(schedule_file)
  df_filtered = filter_schedule(schedule_file_path=schedule_file, start_time=(now_china,'le'), output_type='dataframe')
  df_filtered['MemberID'] = df_filtered['MemberID'].astype(str)
  task_times = df_filtered.groupby('Task').agg({'Start': 'min', 'End': 'max', 'MemberID': lambda x: ','.join(x)})

  # Ensure the generated schedule does not change the finished / ongoing tasks time
  for index in task_times.index:
    task_id = int(index.split()[1])
    for task in all_tasks:
      if task.id == task_id:
        start_time = task_times.loc[index, 'Start']
        dt = datetime.combine(datetime.min, start_time)
        end_time = dt + timedelta(minutes=task.estimated_duration)

        task.time_of_day = [(start_time, end_time)]
        task.members = list(map(int,task_times.loc[index, 'MemberID'].split(',')))
        task.roles = []
      else:
        if f'Task {str(task.id)}' not in list(df_filtered['Task'].unique()):
          for i, timeslot in enumerate(task.time_of_day):
            start_time = timeslot[0]
            end_time = timeslot[1]
            if timeslot[0] < now_china:
              task.time_of_day[i] = (now_china, end_time)

  model = Optimizer(all_tasks, all_members)
  model.schedule_tasks(timeout=timeout)
  print('The optimized cost is: ', model.optimized_cost)
  model.export_schedule(output_location)
  
  return model.optimized_cost


In [16]:
tasks, members = get_members_and_tasks()

In [17]:
tasks

[Task(id=0, estimated_duration=30, location=['Parking Lot Near Loading Bay'], description='JacobMakeup', dependencies=[], time_of_day=[(datetime.time(0, 0), datetime.time(23, 59))], members=[0], roles=[(2, 1)]),
 Task(id=1, estimated_duration=75, location=['Parking Lot Near Loading Bay'], description='SetupLightingParkingLot', dependencies=[], time_of_day=[(datetime.time(0, 0), datetime.time(23, 59))], members=[], roles=[(1, 5), (4, 2)]),
 Task(id=2, estimated_duration=75, location=['Parking Lot Near Loading Bay'], description='VickyMakeup', dependencies=[], time_of_day=[(datetime.time(0, 0), datetime.time(23, 59))], members=[1], roles=[(2, 1)]),
 Task(id=3, estimated_duration=60, location=['Parking Lot Near Loading Bay'], description='JacobShot', dependencies=[(0, 5), 1], time_of_day=[(datetime.time(0, 0), datetime.time(23, 59))], members=[0], roles=[(1, 5), (4, 2)]),
 Task(id=4, estimated_duration=60, location=['The Galleria', 'Studio'], description='VickyPhotoshoot', dependencies=[2

In [28]:
dynamic_schedulling(tasks=[Task(id=16, estimated_duration=60, location=['The Street'], description='SetupLightingStreet', dependencies=[], time_of_day=[(time(0, 0), time(23, 59))], members=[], roles=[(1, 5), (4, 2)])], 
current_time=time(14,40))

C:\Users\nicho\AppData\Local\Temp\ipykernel_76348\1424745392.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Start'] = pd.to_datetime(df['Start']).dt.time
C:\Users\nicho\AppData\Local\Temp\ipykernel_76348\1424745392.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['End'] = pd.to_datetime(df['End']).dt.time


4
The optimized cost is:  33855.0
Solution 1 saved to C:\Users\nicho\OneDrive - HKUST Connect\Python\Github\production_scheduler\nicholas\output


33855.0